In [1]:
# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import copy
import math
import random
import time


# Loading trace : Needs to expanded into 4K chunks
path = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few'
all_files = glob.glob(os.path.join(path, "synthetic_dataprep_B_with_deathtime_new_1.0GB_SSD.csv"))

f = all_files[0]  # Change the file name as required
print("File " + str(f))
cols = ['LBA','Deathtime_RWI']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
df['Deathtime_RWI'] = df['Deathtime_RWI'].replace(np.NaN, -1)
lba_list = df['LBA'].tolist()
deathtime_list = df['Deathtime_RWI'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

File C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\synthetic_dataprep_B_with_deathtime_new_1.0GB_SSD.csv
Min LBA in the dataset : 0
Max LBA in the dataset : 262144
Number of unique LBAs in the data : 262145
Number of IO Accesses : 1048580


In [2]:
def round_decimals_up(number,decimals):
    """
    Returns a value rounded up to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor


# SSD specifications
num_page_addresses = len(Counter(df['LBA']))
page_size = 4096
page_per_block = 64    
GB = 1024*1024*1024
SSD_size = num_page_addresses*page_size
SSD_size_GB_normal = round_decimals_up(SSD_size/GB,1)
over_provisioning_ratio = 0.25 
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))
SSD_size_full = round_decimals_up((1 + over_provisioning_ratio)*SSD_size_GB_normal,1)
print("SSD Capacity (Available in GB) :",SSD_size_GB_normal)
print("SSD Capacity (Total in GB)     :",SSD_size_full)

SSD Capacity (Available in GB) : 1.1
SSD Capacity (Total in GB)     : 1.4


In [3]:
# SSD specifications
GB = 1024*1024*1024
ssd_capacity = SSD_size_full *GB

# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)

block_struct = {}
for x in free_list_block:
    start_lba = x
    write_ptr=0
    invalid_pages=0
    death_time = 0
    valid_bitmap = []
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)

    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    block_struct[start_lba]=segment
    

print("Total number of Blocks created: ", len(block_addresses))
print("Total number of Pages created:  ", len(page_addresses))
print(str(len(block_struct)) + " 4K blocks Initialized" )

GC_THRESHOLD = int(0.25*len(block_addresses))
if(len(page_addresses)*0.75 < len(Counter(df['LBA']))):
    print("WARNING...! Not enough blocks. Need to increase SSD Size")
else:
    print("Looks good.Go ahead!")

Total number of Blocks created:  5671
Total number of Pages created:   362907
5671 4K blocks Initialized
Looks good.Go ahead!


In [4]:
def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][2][prev_page] = False                          # Setting bitmap to False
    block_struct[prev_block][1] = block_struct[prev_block][2].count(True)     # Setting invalid pages
    L2P.pop(lba)


    
#map LBA to phys
def map_lba(lba,deathtime, block_IO_burst):
    # Finding which block to add the LBA
    block_map={}
    found = False
    for x in block_IO_burst:
        # If the death Time has passed, make it priority 
        if(block_struct[x][5] == True):
            block_select = x
            found = True 
        else:
            block_map[x]= block_struct[x][4]
    # Finding the block with closest death time 
    if(found !=True):
        delta = max(deathtime_range_list)*100
        keys = list(block_map.keys())
        block_select = -1
        for x in keys:
            tmp = abs(block_map[x] - deathtime)
            if(tmp < delta):
                delta = tmp
                block_select = x
    
    
    # Block Found, now updating block          
    phys_addr = block_struct[block_select][0] + (block_struct[block_select][3])
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[block_select][2][block_struct[block_select][3]] = True             # Setting Bitmap
    block_struct[block_select][1] = block_struct[block_select][2].count(True)            # Setting invalid pages
    block_struct[block_select][3] = block_struct[block_select][3] + 1               # Increasing Write pointer

    

#check if we need to close/open block. Do not perform GC if we are already
def check_GC (block_IO_burst, in_gc):
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] == page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    if(len(free_list_block) == 0):
        print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
   # Checking if GC is needed
    elif (len(free_list_block) <= GC_THRESHOLD):
        # Checking if GC is already going on
        if(in_gc != True):
            in_gc = do_greedy_gc(block_IO_burst,in_gc) 
    return block_IO_burst




def do_greedy_gc(block_IO_burst,in_gc):
#     print(counter)
    in_gc = True
    gc_writes = 0 
    min_val = float('inf')
    for x in closed_blocks:              
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
            gc_blk = x
        # For each closed block, check phys_addr: If valid bitmap is True (data is valid), copy to OP capacity
#     print(min_val)
#     print(block_struct[gc_blk])
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        # Updating P2L
        if (phys_addr in P2L):
            gc_lba = P2L[phys_addr]
            P2L.pop(phys_addr)            
        # Updating L2P
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        death_time = block_struct[prev_block][4]                      # Getting current death_time for the block
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(gc_lba)
            gc_writes = gc_writes + 1
            #check if we need to get a new block
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(gc_lba,death_time,block_IO_burst)
            block_IO_burst = decrease_death_time(block_IO_burst)  
    
                
    if(gc_writes > 64):
        print("GC writes not as expected", gc_writes)
    total_gc_writes[0] = total_gc_writes[0] + gc_writes
    
    invalid_pages = 0
    valid_bitmap = []
    write_ptr = 0
    death_time = 0
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)
            
    # Reseting the GC blk and add to free_block_list
    block_struct[gc_blk]= [gc_blk,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    closed_blocks.remove(gc_blk)
    free_list_block.append(gc_blk)
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] >= page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    in_gc = False
    return in_gc

def decrease_death_time(block_IO_burst):
    # Decreasing Death Time for each block
    for x in block_IO_burst:
        # If death time passed, make the block priority 
        # Priority means: All subsequent IOs will be added to this block here until full
        if (block_struct[x][4] <= 0):
            block_struct[x][5] = True
            priority_writes[0] = priority_writes[0] + 1
        else:
            block_struct[x][4] = block_struct[x][4] - 1  
    return block_IO_burst


In [5]:
# Setting global parameters
# Initalizing Starting Free Blocks..
global gc_writes
global in_gc 


GC_THRESHOLD = 0.2*len(block_addresses)
num_cur_blocks_open = 5        # Hyperparameter

L2P = {}
P2L = {}
closed_blocks = []
cur_blocks_open = []
lba_burst = []
deathtime_range_list = []
interval = float(100/num_cur_blocks_open)
gc_writes = 0
in_gc = False
death_time_passed = []

for x in range(num_cur_blocks_open):
    deathtime_range_list.append(int(np.percentile(deathtime_list, (x+1)*interval)))


block_IO_burst = []
death_time_ranges = []
print("Initalizing Starting Free Blocks...")
for x in range(num_cur_blocks_open):
    block_num = free_list_block.pop(0)                                            # Getting a free block
    block_IO_burst.append(block_num) 
    block_struct[block_num][4] = deathtime_range_list[x]                          # Setting death time
    block_struct[block_num][6] = deathtime_range_list[x]                          # Setting original Death time



Initalizing Starting Free Blocks...


In [6]:
total_gc_writes = []
total_gc_writes.append(0)
priority_writes = []
priority_writes.append(0)
counter = 0

print("Starting Trace..!")
start_time = time.time()
while(counter < len(lba_list)):
    if(counter >100000 and counter%100000==0):
        print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
    lba= int(lba_list[counter])
    death_time = int(deathtime_list[counter])
    if lba in L2P:
        invalidate_lba(lba)
    block_IO_burst = check_GC(block_IO_burst,in_gc)
    map_lba(lba,death_time,block_IO_burst)
    block_IO_burst = decrease_death_time(block_IO_burst)
    counter = counter + 1

end_time = time.time()
run_time = end_time - start_time
print("Execution Time for the FTL :",run_time)
print("Total Number of GC writes :",total_gc_writes[0])
print("Total Number of Priority writes    :",priority_writes[0])

Starting Trace..!
Percentage completed in (%)  : 19.073413568826414
Percentage completed in (%)  : 28.610120353239623
Percentage completed in (%)  : 38.14682713765283
Percentage completed in (%)  : 47.68353392206603
Percentage completed in (%)  : 57.220240706479245
Percentage completed in (%)  : 66.75694749089244
Percentage completed in (%)  : 76.29365427530566
Percentage completed in (%)  : 85.83036105971887
Percentage completed in (%)  : 95.36706784413207
Execution Time for the FTL : 77.27578711509705
Total Number of GC writes : 2120178
Total Number of Priority writes    : 3493452


In [7]:
print('Done...Sanity Check complete!!')

Done...Sanity Check complete!!


In [8]:
gc_writes_baseline=9346492 # From FTL Baseline Run on synthetic trace B
Savings = (gc_writes_baseline- total_gc_writes[0]) /gc_writes_baseline
print(Savings*100)

77.31578864027273
